# Linear Regression with PyTorch built-ins

We're going to use the built-in neural network package on pytorch 

In [4]:
import torch 
import torch.nn as nn
import numpy as np

In [5]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)


In [11]:
from torch.utils.data import TensorDataset

In [12]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]


(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

### Tensordata
*Tensordata* helps us consolidate input and target variables for each instance as a tuple. The first element contains the input, second element of the tuple contains the corresponding target variables. 

### DataLoader
We'll also create a *DataLoader*, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [13]:
from torch.utils.data import DataLoader

In [18]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True) #shuffle equals True --> shuffles rows of a dataset

The dataloader is usually used in a for-in loop. 

In [24]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break # stops after printing first batch



tensor([[ 91.,  88.,  64.],
        [ 73.,  67.,  43.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [102.,  43.,  37.]])
tensor([[ 81., 101.],
        [ 56.,  70.],
        [ 22.,  37.],
        [103., 119.],
        [ 22.,  37.]])


### Defining model using nn.Linear(# predictor variables, # target variables)

In [27]:
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.5516, -0.4918, -0.1162],
        [ 0.5603,  0.3813,  0.4574]], requires_grad=True)
Parameter containing:
tensor([-0.0760, -0.2814], requires_grad=True)


### model.parameters() contains list of weights and biases matrix


In [35]:
list(model.parameters()) #same as printing model.weight and model.bias

[Parameter containing:
 tensor([[ 0.5516, -0.4918, -0.1162],
         [ 0.5603,  0.3813,  0.4574]], requires_grad=True),
 Parameter containing:
 tensor([-0.0760, -0.2814], requires_grad=True)]

### Generating predictions

In [36]:
preds = model(inputs)
preds

tensor([[  2.2415,  85.8313],
        [ -0.5989, 113.5279],
        [-24.7328, 126.0824],
        [ 30.7409,  90.1835],
        [-17.3672, 106.9968],
        [  2.2415,  85.8313],
        [ -0.5989, 113.5279],
        [-24.7328, 126.0824],
        [ 30.7409,  90.1835],
        [-17.3672, 106.9968],
        [  2.2415,  85.8313],
        [ -0.5989, 113.5279],
        [-24.7328, 126.0824],
        [ 30.7409,  90.1835],
        [-17.3672, 106.9968]], grad_fn=<AddmmBackward>)

### Defining loss function using torch.nn.functional 

In [38]:
import torch.nn.functional as F #contains many loss functions 

### F.mse_loss, built in loss function

In [41]:
loss_fn = F.mse_loss

In [42]:
loss  = loss_fn(model(inputs), targets)
print(loss)

tensor(4820.0137, grad_fn=<MseLossBackward>)


## Optimizer

In [49]:
#Define optimizer 
opt = torch.optim.SGD(model.parameters(), lr = 1e-5)


#passing in model.parameters() specifies which parameter matrices need to be updated 
#lr is learning rate 
#other built-in optim methods: Adam, AdaGrad, RMSProp

Let's recollect our steps 

- define the model
- produce predictions
- evaluate your model prediction using loss function 
- optimize loss value by updating weights and bias parameters 


In [ ]:
## Utility function to train the model
def fit(num_epochs, model, loss_fn, opt):
    for epoch in range(num_epochs):
        for xb,yb in train_dl:
            pred =  model(xb)
            loss =  loss_fn(pred,yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
        if (epoch+1) % 10 == 0:
            print('Epoch  [{}/{}],Loss: {:.4f}'.format(epoch+1, num_epochs ))
